In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
print("Working directory:", os.getcwd())
path = os.path.abspath('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
print("Absolute path being used:", path)
print("Exists?", os.path.exists(path))

# # Force the working dir to the 'model' folder
# os.chdir(r"C:\Users\domho\OneDrive\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\model")

Working directory: C:\Users\tobia\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\model
Absolute path being used: C:\Users\tobia\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\data\cleaned_data\infrastructure\BMMS_overview.xlsx
Exists? True


In [3]:
bridges = pd.read_excel('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
roads = pd.read_csv('../data/cleaned_data/infrastructure/_roads3.csv')

In [ ]:
# Step 2: Filter only bridges related to road N1
bridges_n1 = bridges[bridges['road'] == 'N1']

# Ensure column names and data types match for merging or appending
# For example, align the column 'type' in bridges with 'model_type' in roads
bridges_n1 = bridges_n1.rename(columns={"chainage": "chainage", "type": "model_type"})

# Add any necessary default values to match columns in 'roads' DataFrame
bridges_n1['id'] = range(roads['id'].max() + 1, roads['id'].max() + 1 + len(bridges_n1))
bridges_n1['length'] = 0  # Default length for bridges
bridges_n1['model_type'] = "bridge"  # Ensure type is recognized as 'bridge'

# # Step 5: Save the updated roads DataFrame to a CSV file
# roads.to_csv("../data/cleaned_data/infrastructure/updated_roads.csv", index=False)
#
# # Optional: Display a preview of the updated DataFrame
# roads.head(20)

In [4]:
bridges["type"] = bridges["type"].astype(str).str.lower()
b = bridges["type"].unique()
b = b.tolist()
b.append("culvert")
b.append("bridge")
b = np.array(b)
print(b)


['box culvert' 'pc girder bridge' 'pc box' 'rcc girder bridge'
 'slab culvert' 'steel beam & rcc slab' 'arch masonry' 'rcc bridge'
 'truss with rcc slab' 'baily with steel deck' 'truss with steel deck'
 'baily with timber deck' 'truss with timber deck' 'pipe culvert'
 'culvert' 'bridge']


In [5]:
# Rename and reformat the columns
roads["length"] = roads["chainage"].diff().abs().fillna(0)
roads["type"] = roads["type"].astype(str).str.lower()

# Update 'type' column values to standardize naming
roads.loc[roads["type"].isin(b), "type"] = "bridge"
roads.loc[roads["type"] != "bridge", "type"] = "link"

# Generate unique IDs for each road segment
roads["id"] = roads.index
roads["id"] = (roads["id"] + 1_000_000).astype(int)

# Correct assignment for model_type using .loc for single-element assignment
roads.loc[0, "type"] = "source"  # Assign 'source' to the first row
roads.loc[len(roads.index) - 1, "type"] = "sink"  # Assign 'sink' to the last row

# Rename column 'type' to 'model_type'
roads = roads.rename(columns={
    "type": "model_type",
})

# Drop unnecessary columns
roads = roads.drop('chainage', axis=1)
roads = roads.drop('lrp', axis=1)
roads = roads.drop('gap', axis=1)

# Save the formatted data to a CSV file
roads.to_csv("../data/cleaned_data/infrastructure/roads_for_model.csv", index=False)

# Display the first 40 rows for verification
roads.head(40)

,road,lat,lon,model_type,name,length,id
0,N1,23.706028,90.443333,source,Start of Road after Jatrabari Flyover infront...,0.000,1000000
1,N1,23.702917,90.450417,bridge,Box Culvert,0.814,1000001
2,N1,23.702778,90.450472,link,Intersection with Z1101,0.008,1000002
3,N1,23.702139,90.451972,link,Km post missing,0.178,1000003
4,N1,23.697889,90.460583,link,Km post missing,1.000,1000004
5,N1,23.697361,90.461667,bridge,Box culvert,0.130,1000005
6,N1,23.693833,90.469138,link,Km post missing,0.870,1000006
7,N1,23.693611,90.478777,link,Km post missing,1.000,1000007
8,N1,23.693805,90.480527,link,Road to Narayanganj(R111),0.175,1000008
9,N1,23.694750,90.488500,link,Km post missing,0.825,1000009


In [6]:
# Step 3: Append it to the roads DataFrame
roads = pd.concat([roads, bridges_n1], ignore_index=True)

# Step 4: Sort the updated DataFrame by chainage
roads = roads.sort_values(by='chainage').reset_index(drop=True)

,road,lat,lon,model_type,name,length,id,km,LRPName,condition,...,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,EstimatedLoc
0,N1,23.698739,90.458861,bridge,.,0.0,1051348,1.800,LRP001a,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.800,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,interpolate
1,N1,23.694664,90.487775,bridge,.,0.0,1051349,4.925,LRP004b,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,interpolate
2,N1,23.705060,90.523214,bridge,Kanch pur Bridge.,0.0,1051350,8.976,LRP008b,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,interpolate
3,N1,23.705060,90.523214,bridge,KANCHPUR PC GIRDER BRIDGE,0.0,1051814,8.976,LRP008b,C,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,14.65,1986.0,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,interpolate
4,N1,23.696400,90.535099,bridge,KATCHPUR BOX CULVERT,0.0,1051844,10.543,LRP010a,B,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.543,21.70,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
5,N1,23.694391,90.537574,bridge,NOYAPARA CULVERT,0.0,1051351,10.880,LRP010b,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
6,N1,23.694302,90.537707,bridge,ADUPUR CULVERT,0.0,1051352,10.897,LRP010c,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
7,N1,23.692360,90.540918,bridge,NAYABARI KASPUR BOX CULVERT,0.0,1051353,11.296,LRP011a,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
8,N1,23.690912,90.545462,bridge,NAYABARI BOX CULVERT,0.0,1051845,11.808,LRP011c,B,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.808,22.40,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
9,N1,23.688412,90.548559,bridge,KHAS PARA BOX CULVERT,0.0,1051354,12.239,LRP012a,A,...,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.239,21.00,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,interpolate
